# Telegram Bot

In [1]:
#importing necessary libraries
import telebot
import pandas as pd
from telebot import types
from sklearn.preprocessing import OneHotEncoder
import re
from sklearn.metrics.pairwise import cosine_similarity

data=pd.read_csv('Cleaned_data.csv')

# Replace this with your actual bot token
bot = telebot.TeleBot("7445378294:AAFS_LavGa5y4Yplx55nYClyS61jDCENzbQ")

In [2]:
#introducing and starting the bot 
@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, '''Greetings, seeker of serenity and wisdom! I am YourSpiritualGuide, your own personal spiritual guide bot on this journey through the sacred and serene landscapes of Delhi. With a city rich in history, culture, and spirituality, there are countless hallowed grounds waiting to be discovered.
    🕌 Whether you seek the tranquility of ancient temples, the serenity of mosques, the peace of churches, or the quiet contemplation of gurudwaras, I am here to guide you.
    📍 Simply share your preferences, be it by religion, region, or specific spiritual subcategories, and I will illuminate the path to the divine destinations that resonate with your soul. 
    The supported religions are Hinduism, Islam, Jainism, Sikhism and Christianity.
    The supported regions are North, South, East, West, Central and Old Delhi.
    The supported subcategories are Hindu Temple, Jain Temple, Church, Gurdwara, Mosque and Tomb.
    Use the following commands to get started: 
    1) I practice (Type your Religion). Please recommend religious places in Delhi based on my preference. 
    2) I wish to visit a (Type your Subcategory) in Delhi. Please recommend religious sites in Delhi based on my preference.
    3) I follow (Type your Religion) and I want to visit relevant religious sites in (Type your Region preference).
    4) I would like to visit a (Type your Subcategory) located in (Type your Region Preference) Delhi.''')

In [3]:
#This function is used to break the response into chunks as the response has a maximum limit 
def chunk_message(message, max_length=4096):
    for i in range(0, len(message), max_length):
        yield message[i:i + max_length]

### To give a response to the user based on his custom religion preference

In [4]:
def recommend_places(religion):
    # Filter the data based on the provided religion
    filtered_data = data[data['Religion'] == religion]
    
    # Extract names of religious places
    places = filtered_data.to_dict('records')
    
    return places

@bot.message_handler(func=lambda message: bool(re.search(r'\b(I\s+practice\s+(?:Hinduism|Islam|Christianity|Jainism|Sikhism))', message.text, re.IGNORECASE)))
def recommend(message):
    try:
        # Extract religion from user's message
        match = re.search(r'\b(I\s+practice\s+(Hinduism|Islam|Christianity|Jainism|Sikhism))', message.text, re.IGNORECASE)
        if match:
            religion = match.group(1).split()[-1]  # Extracting the religion from the matched group
        else:
            bot.reply_to(message, "Please specify a valid religion (Hinduism, Islam, Christianity, Jainism, Sikhism).")
            return
        
        # Get the recommendations
        places = recommend_places(religion)
        
        if places:
            response = f"Here's a list of recommended religious sites in Delhi for {religion}:\n"
            for place in places:
                response += f"\nName: {place['Names']}\n"
                response += f"Location: {place['Location']}\n"
                response += f"Description: {place['Description']}\n"
                response += f"How to Reach: {place['How to Reach?']}\n"
                response += f"Opening and Closing Hours: {place['Opening and Closing Hours']}\n"
                response += "----------------------\n"
        else:
            response = f"No places found for {religion}."
        
        # Send the response in chunks
        for chunk in chunk_message(response):
            bot.reply_to(message, chunk)
    except Exception as e:
        bot.reply_to(message, str(e))

#### The next step is done as the Hindu Temple and Jain Temple are two separate words and to extract both the word's from user's message I am combining them into one.

In [5]:
data['Subcategory'] = data['Subcategory'].str.replace('Hindu\s+Temple', 'HinduTemple', regex=True)

In [6]:
data['Subcategory'] = data['Subcategory'].str.replace('Jain\s+Temple', 'JainTemple', regex=True)

### To give a response to the user based on his custom subcategory preference

In [7]:
def recommend_places_subcategory(subcategory):
    # Filter the data based on the provided religion
    filtered_data1 = data[data['Subcategory'] == subcategory]
    
    # Extract names of religious places
    places1 = filtered_data1.to_dict('records1')
    
    return places1

@bot.message_handler(func=lambda message: bool(re.search(r'\b(I\s+wish\s+to\s+visit\s+a\s+(?:HinduTemple|Mosque|Cathedral|Church|Gurdwara|Tomb|JainTemple))', message.text, re.IGNORECASE)))
def recommend2(message):
    try:
        # Extract religion from user's message
        match = re.search(r'\b(I\s+wish\s+to\s+visit\s+a\s+(HinduTemple|Mosque|Church|Gurdwara|Tomb|Cathedral|JainTemple))', message.text, re.IGNORECASE)
        if match:
            subcategory = match.group(1).split()[-1]
        else:
            bot.reply_to(message, "Please specify a valid subcategory (Hindu Temple,Mosque,Church, Cathedral, Gurdwara,Tomb,Jain Temple).")
            return
        
        # Get the recommendations
        places1 = recommend_places_subcategory(subcategory)
        
        if places1:
            response = f"Here's a list of recommended {subcategory}s:\n"
            for place in places1:
                response += f"\nName: {place['Names']}\n"
                response += f"Location: {place['Location']}\n"
                response += f"Description: {place['Description']}\n"
                response += f"How to Reach: {place['How to Reach?']}\n"
                response += f"Opening and Closing Hours: {place['Opening and Closing Hours']}\n"
                response += "----------------------\n"
        else:
            response = f"No places found for {subcategory}."
        
        # Send the response in chunks
        for chunk in chunk_message(response):
            bot.reply_to(message, chunk)
    except Exception as e:
        bot.reply_to(message, str(e))

### To give a response to the user based on his custom religion and region preference

In [8]:
def recommend_places_relregion(religion, region):
    # Filter the data based on the provided religion and region
    filtered_data2 = data[(data['Religion'] == religion) & (data['Region'].str.contains(region, case=False))]
    # Extract names of religious places
    places2 = filtered_data2.to_dict('records2')
    return places2

@bot.message_handler(func=lambda message: bool(re.search(r'\b(I\s+follow\s+(?:Hinduism|Islam|Christianity|Jainism|Sikhism)\b.*\b(?:North|South|West|East|Central|Old)\s+Delhi\b)', message.text, re.IGNORECASE)))
def recommend3(message):
    try:
        # Extract religion and region from user's message
        match = re.search(r'\bI\s+follow\s+(Hinduism|Islam|Christianity|Jainism|Sikhism)\b.*\b(North|South|West|East|Central|Old)\s+Delhi\b', message.text, re.IGNORECASE)
        if match:
            religion = match.group(1)  # Extracting the religion from the matched group
            region = match.group(2) + " Delhi"  # Extracting the region from the matched group
        else:
            bot.reply_to(message, "Please specify a valid religion (Hinduism, Islam, Christianity, Jainism, Sikhism) and region (North, South, West, East, Central, Old Delhi).")
            return
        
        # Get the recommendations
        places2 = recommend_places_relregion(religion, region)
        
        if places2:
            response = f"Here's a list of recommended religious places for {religion} in {region}:\n"
            for place in places2:
                response += f"\nName: {place['Names']}\n"
                response += f"Location: {place['Location']}\n"
                response += f"Description: {place['Description']}\n"
                response += f"How to Reach: {place['How to Reach?']}\n"
                response += f"Opening and Closing Hours: {place['Opening and Closing Hours']}\n"
                response += "----------------------\n"
        else:
            response = f"No places found for {religion} in {region}."
        
        # Send the response in chunks
        for chunk in chunk_message(response):
            bot.reply_to(message, chunk)
    except Exception as e:
        bot.reply_to(message, str(e))

### To give a response to the user based on his custom subcategory and region preference

In [9]:
def recommend_places_subcatregion(subcategory, region):
    # Filter the data based on the provided religion and region
    filtered_data3 = data[(data['Subcategory'] == subcategory) & (data['Region'].str.contains(region, case=False))]
    # Extract names of religious places
    places3 = filtered_data3.to_dict('records3')
    return places3

@bot.message_handler(func=lambda message: bool(re.search(r'\b(I\s+would\s+like\s+to\s+visit\s+a\s+(?:HinduTemple|Mosque|Church|Gurdwara|Tomb|JainTemple)\b.*\b(?:North|South|West|East|Central|Old)\s+Delhi\b)', message.text, re.IGNORECASE)))
def recommend4(message):
    try:
        # Extract religion and region from user's message
        match = re.search(r'\bI\s+would\s+like\s+to\s+visit\s+a\s+(HinduTemple|Mosque|Church|Gurdwara|Tomb|JainTemple)\b.*\b(North|South|West|East|Central|Old)\s+Delhi\b', message.text, re.IGNORECASE)
        if match:
            subcategory = match.group(1)  # Extracting the religion from the matched group
            region = match.group(2) + " Delhi"  # Extracting the region from the matched group
        else:
            bot.reply_to(message, "Please specify a valid subcategory (Hindu Temple, Jain Temple, Church, Gurdwara, Mosque, Tomb) and region (North, South, West, East, Central, Old Delhi).")
            return
        
        # Get the recommendations
        places3 = recommend_places_subcatregion(subcategory, region)
        
        if places3:
            response = f"Here's a list of recommended religious places for {subcategory} in {region}:\n"
            for place in places3:
                response += f"\nName: {place['Names']}\n"
                response += f"Location: {place['Location']}\n"
                response += f"Description: {place['Description']}\n"
                response += f"How to Reach: {place['How to Reach?']}\n"
                response += f"Opening and Closing Hours: {place['Opening and Closing Hours']}\n"
                response += "----------------------\n"
        else:
            response = f"No places found for {subcategory} in {region}."
        
        # Send the response in chunks
        for chunk in chunk_message(response):
            bot.reply_to(message, chunk)
    except Exception as e:
        bot.reply_to(message, str(e))

In [ ]:
# to start the bot 
bot.polling()